What's been done here?
1. Add HPA & disease data to gene,tissue parameter dataframe.
2. Normalized all the features.
3. Also disease labels added to just gtex params as well.

Data from - 
Disease data -  https://www.sciencedirect.com/science/article/pii/S2405471217302909?via%3Dihub
HPA data - https://www.proteinatlas.org/download/normal_tissue.tsv.zip

code referred from - 
HPA ref - http://localhost:4321/notebooks/azure-backup-notebooks/GTF_Manuscript/HPA_Info.ipynb
Disease mapping ref - http://localhost:4321/notebooks/azure-backup-notebooks/GTF_Manuscript/MS_N5.ipynb

End product - Looks like below - Available at 

/scratch1/gowthami/GTF/GTF_Project_v2/data/gt_param_df_labelled.csv
/scratch1/gowthami/GTF/GTF_Project_v2/data/gtex_params_label.csv

What more needs to be done?
1. 

Notes:

- There is no clear HPA data for certain gene-tissue combinations, so -1 is the placeholder for level HPA for such cases
- And following is the logic used for determining Positive & Negative labels

    if (row['Pos']>=1)&(row['Level_enc']>0) :
        label_list.append('P')
    elif (row['Pos']>=1)&(row['Level_enc']==0):
        label_list.append('U')
    elif (row['Pos']>=1)&(row['Level_enc']<0):
        label_list.append('P')
    elif (row['Pos']==0)&(row['Level_enc']>0) :
        label_list.append('U')
    elif (row['Pos']==0)&(row['Level_enc']==0):
        label_list.append('N')
    else:
        label_list.append('U')


In [2]:
gt_param_df = pd.read_csv('/fs/cbcb-scratch/gowthami/GTF/GTF_Project_v2/data/gt_param_df_labl_50thre_norm.csv')
gt_param_df.head(4)

NameError: name 'pd' is not defined

In [80]:
gtex_params_label.head()

,G_T,Gene,Tissue,MeanExp,SDofExp,ZScore,MinExp,MaxExp,MedianExp,Breadth,CoeffOfVar,MedAbsDev,Pos
0,A1BG_Adipose Tissue,A1BG,Adipose Tissue,3.408423,2.116964,-0.121740,0.43950,27.860,2.9600,0.490590,0.621098,1.430711,0.0
1,A1BG_Adrenal Gland,A1BG,Adrenal Gland,1.605872,1.084509,-0.150274,0.28480,10.690,1.3755,0.073684,0.675340,0.573989,0.0
2,A1BG_Bladder,A1BG,Bladder,1.734455,0.725400,-0.148239,1.01100,3.345,1.4870,0.090909,0.418230,0.487776,0.0
3,A1BG_Blood,A1BG,Blood,3.159893,2.716399,-0.125674,0.08606,13.200,2.0910,0.355680,0.859649,1.601210,0.0
4,A1BG_Blood Vessel,A1BG,Blood Vessel,5.657147,2.907591,-0.086144,0.87830,17.070,5.2300,0.798467,0.513968,2.941483,0.0


In [1]:
import pandas as pd
import csv
import numpy as np
import scipy
import pickle
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=40,20
import matplotlib.mlab as mlab
from IPython.display import display
pd.options.display.max_columns = None
from numpy import random
from statsmodels import robust
from scipy.stats import spearmanr
from scipy.stats import pearsonr
import itertools
import networkx as nx
import datetime
import io
import random
import warnings
import json
import requests
import time

In [6]:
gt_param_df = pd.read_csv('./data_generated/gt_param_df.csv')
gt_param_df.head(2)

,Gene,Tissue,MeanExp,SDofExp,ZScore,MedianExp,Breadth,CV,MAD,Clustering_coeff,IsKinase,IsTF,NeighborList,NumOfKinaseNbs,NumOfNeighbors,NumOfTFNeighbors,Centrality_coeff,NeighborMeanBreadth,NeighborMeanZscore,NeighborMedianBreadth,NeighborMedianZscore,Description
0,ENSG00000187634,Stomach,4.720096,4.267317,-0.036851,3.423000,0.763231,0.904074,2.489289,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,7,2,0.000001,0.296060,-0.366241,0.261838,-0.380078,SAMD11
1,ENSG00000188976,Stomach,44.847466,15.854806,-0.412520,42.380001,0.220056,0.353527,14.366420,0.253545,0,0,"['ENSG00000111676', 'ENSG00000166333', 'ENSG00...",2,110,15,0.000094,0.349456,-0.245143,0.309192,-0.260509,NOC2L


In [3]:
gt_param_df[gt_param_df['Gene']=='ENSG00000187634']

,Gene,Tissue,MeanExp,SDofExp,ZScore,MedianExp,Breadth,CV,MAD,Clustering_coeff,IsKinase,IsTF,NeighborList,NumOfKinaseNbs,NumOfNeighbors,NumOfTFNeighbors,Centrality_coeff,NeighborMeanBreadth,NeighborMeanZscore,NeighborMedianBreadth,NeighborMedianZscore,Description
0,ENSG00000187634,Stomach,4.720096,4.267317,-0.036851,3.423000,0.763231,0.904074,2.489289,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,7,2,1.209951e-06,0.296060,-0.366241,0.261838,-0.380078,SAMD11
92,ENSG00000187634,Prostate,21.850901,17.702188,1.846739,15.920000,0.995918,0.810135,11.890471,0.100000,0,0,"['ENSG00000121060', 'ENSG00000115239', 'ENSG00...",0,5,2,6.463013e-07,0.765714,0.538349,0.844898,0.559634,SAMD11
187,ENSG00000187634,Salivary Gland,0.967051,0.841292,-0.449510,0.702250,0.111111,0.869956,0.501713,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,7,2,1.279273e-06,0.320106,-0.271974,0.308642,-0.312886,SAMD11
285,ENSG00000187634,Testis,7.346544,2.507698,0.251936,6.864000,0.997230,0.341344,2.299515,0.100000,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,5,2,7.092571e-07,0.541828,0.394516,0.529086,-0.173701,SAMD11
378,ENSG00000187634,Brain,1.499870,1.829619,-0.390925,0.849600,0.222180,1.219851,0.688521,0.000000,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,4,1,5.735608e-07,0.283497,-0.232846,0.169758,-0.427209,SAMD11
473,ENSG00000187634,Liver,2.069460,1.877869,-0.328297,1.632000,0.384956,0.907420,1.056799,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,7,2,1.532388e-06,0.072693,-0.893332,0.004425,-1.045345,SAMD11
565,ENSG00000187634,Muscle,0.525075,0.828505,-0.498107,0.282000,0.047323,1.577880,0.217646,0.000000,0,0,"['ENSG00000115239', 'ENSG00000025770', 'ENSG00...",0,3,0,4.319534e-08,0.015774,-1.149848,0.008717,-1.020626,SAMD11
657,ENSG00000187634,Blood Vessel,9.365276,10.441980,0.473903,5.792000,0.824719,1.114968,5.696157,0.000000,0,0,"['ENSG00000128591', 'ENSG00000115239', 'ENSG00...",0,5,1,4.138612e-07,0.789363,0.289884,0.796255,0.136600,SAMD11
748,ENSG00000187634,Ovary,4.795195,9.094339,-0.028593,1.799500,0.483333,1.896553,1.666741,0.000000,0,0,"['ENSG00000128591', 'ENSG00000143889', 'ENSG00...",0,3,0,1.341963e-07,0.759259,0.269841,0.688889,0.027629,SAMD11
842,ENSG00000187634,Uterus,15.662472,21.725756,1.166300,11.290001,0.964789,1.387122,8.628747,0.066667,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,6,2,2.766941e-06,0.920188,0.652182,0.950704,0.738003,SAMD11


In [7]:
HPA_to_Gtex_mapping = {
    'adrenal gland' : 'Adrenal Gland',
    'breast' : 'Breast',
    'heart muscle' : 'Heart',
    'liver' : 'Liver',
    'pancreas' : 'Pancreas',
    'ovary' : 'Ovary',
    'testis' : 'Testis',
    'lung' : 'Lung',
    'thyroid gland' : 'Thyroid',
    'pituitary gland' : 'Pituitary',
    'prostate' : 'Prostate',
    'endometrium 1': 'Uterus',
    'endometrium 2' : 'Uterus',
    'skin' : 'Skin',
    'skin 1' : 'Skin',
    'skin 2' :'Skin',
    'skeletal muscle' : 'Muscle',
    'smooth muscle' : 'Muscle',
    'cerebral cortex': 'Brain',
    'cerebellum' : 'Brain',
    'hippocampus' : 'Brain',
    'cervix, uterine' : 'Cervix Uteri',
    'colon' : 'Colon',
    'esophagus' : 'Esophagus',
    'fallopian tube' : 'Fallopian Tube',
    'hypothalamus' : 'Brain',
    'kidney' : 'Kidney',
    'salivary gland' : 'Salivary Gland',
    'small intestine' : 'Small Intestine',
    'stomach 1' : 'Stomach',
    'stomach 2' : 'Stomach',
    'vagina' : 'Vagina'
    
}

In [8]:
tis_sl = HPA_to_Gtex_mapping.keys()
tissue_atl = pd.read_csv('./data/normal_tissue.tsv',sep='\t')
tissue_atl = tissue_atl[tissue_atl['Tissue'].isin(tis_sl)]
tissue_atl = tissue_atl[tissue_atl['Reliability'].isin(['Enhanced', 'Supported','Approved'])]
tissue_atl['Gtex_Tissue'] = tissue_atl['Tissue'].map(HPA_to_Gtex_mapping)
print(tissue_atl.shape)
tissue_atl[(tissue_atl['Gtex_Tissue'] == 'Uterus') & (tissue_atl['Gene name'] == 'TSPAN6')]

(550041, 7)


,Gene,Gene name,Tissue,Cell type,Level,Reliability,Gtex_Tissue
24,ENSG00000000003,TSPAN6,endometrium 1,cells in endometrial stroma,Not detected,Approved,Uterus
25,ENSG00000000003,TSPAN6,endometrium 1,glandular cells,High,Approved,Uterus
26,ENSG00000000003,TSPAN6,endometrium 2,cells in endometrial stroma,Not detected,Approved,Uterus
27,ENSG00000000003,TSPAN6,endometrium 2,glandular cells,High,Approved,Uterus


In [9]:
rel_map = {
    'High' : 1, 
    'Medium' : 0.75, 
    'Low': 0.5, 
    'Not detected' : 0
}

tissue_atl['Level_enc'] = tissue_atl['Level'].map(rel_map)
tissue_atl_short = tissue_atl.groupby(['Gene', 'Gtex_Tissue'])['Level_enc'].mean().to_frame().reset_index()
tissue_atl_short = tissue_atl_short.rename(columns={'Gtex_Tissue':'Tissue'})
tissue_atl_short[tissue_atl_short['Gene']=='ENSG00000000003']

,Gene,Tissue,Level_enc
0,ENSG00000000003,Adrenal Gland,0.000000
1,ENSG00000000003,Brain,0.083333
2,ENSG00000000003,Breast,0.333333
3,ENSG00000000003,Cervix Uteri,1.000000
4,ENSG00000000003,Colon,0.250000
5,ENSG00000000003,Esophagus,1.000000
6,ENSG00000000003,Fallopian Tube,1.000000
7,ENSG00000000003,Heart,0.000000
8,ENSG00000000003,Kidney,0.375000
9,ENSG00000000003,Liver,0.375000


In [10]:
gt_param_df = pd.merge(gt_param_df, tissue_atl_short,  how='left', 
                       left_on=['Gene','Tissue'], right_on = ['Gene','Tissue'])

print(gt_param_df.shape)
gt_param_df.head(2)

(453455, 23)


,Gene,Tissue,MeanExp,SDofExp,ZScore,MedianExp,Breadth,CV,MAD,Clustering_coeff,IsKinase,IsTF,NeighborList,NumOfKinaseNbs,NumOfNeighbors,NumOfTFNeighbors,Centrality_coeff,NeighborMeanBreadth,NeighborMeanZscore,NeighborMedianBreadth,NeighborMedianZscore,Description,Level_enc
0,ENSG00000187634,Stomach,4.720096,4.267317,-0.036851,3.423000,0.763231,0.904074,2.489289,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,7,2,0.000001,0.296060,-0.366241,0.261838,-0.380078,SAMD11,NaN
1,ENSG00000188976,Stomach,44.847466,15.854806,-0.412520,42.380001,0.220056,0.353527,14.366420,0.253545,0,0,"['ENSG00000111676', 'ENSG00000166333', 'ENSG00...",2,110,15,0.000094,0.349456,-0.245143,0.309192,-0.260509,NOC2L,0.0


In [11]:
gt_param_df.isna().sum()

Gene                          0
Tissue                        0
MeanExp                       0
SDofExp                       0
ZScore                        0
MedianExp                     0
Breadth                       0
CV                            0
MAD                           0
Clustering_coeff              0
IsKinase                      0
IsTF                          0
NeighborList                  0
NumOfKinaseNbs                0
NumOfNeighbors                0
NumOfTFNeighbors              0
Centrality_coeff              0
NeighborMeanBreadth           0
NeighborMeanZscore            0
NeighborMedianBreadth         0
NeighborMedianZscore          0
Description                   0
Level_enc                235997
dtype: int64

In [12]:
gt_param_df = gt_param_df.fillna(value=-1)
gt_param_df.head(2)

,Gene,Tissue,MeanExp,SDofExp,ZScore,MedianExp,Breadth,CV,MAD,Clustering_coeff,IsKinase,IsTF,NeighborList,NumOfKinaseNbs,NumOfNeighbors,NumOfTFNeighbors,Centrality_coeff,NeighborMeanBreadth,NeighborMeanZscore,NeighborMedianBreadth,NeighborMedianZscore,Description,Level_enc
0,ENSG00000187634,Stomach,4.720096,4.267317,-0.036851,3.423000,0.763231,0.904074,2.489289,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,7,2,0.000001,0.296060,-0.366241,0.261838,-0.380078,SAMD11,-1.0
1,ENSG00000188976,Stomach,44.847466,15.854806,-0.412520,42.380001,0.220056,0.353527,14.366420,0.253545,0,0,"['ENSG00000111676', 'ENSG00000166333', 'ENSG00...",2,110,15,0.000094,0.349456,-0.245143,0.309192,-0.260509,NOC2L,0.0


In [13]:
gt_param_df.isna().sum()

Gene                     0
Tissue                   0
MeanExp                  0
SDofExp                  0
ZScore                   0
MedianExp                0
Breadth                  0
CV                       0
MAD                      0
Clustering_coeff         0
IsKinase                 0
IsTF                     0
NeighborList             0
NumOfKinaseNbs           0
NumOfNeighbors           0
NumOfTFNeighbors         0
Centrality_coeff         0
NeighborMeanBreadth      0
NeighborMeanZscore       0
NeighborMedianBreadth    0
NeighborMedianZscore     0
Description              0
Level_enc                0
dtype: int64

In [14]:
len(gt_param_df['Gene'].drop_duplicates().tolist())

16387

In [15]:
disease_list = pd.read_excel('./data/1-s2.0-S2405471217302909-mmc3.xlsx',skiprows=4)
disease_list.head(2)

,gene,mim.ids,mim.titles,Adipose Tissue,Adrenal Gland,Blood,Blood Vessel,Brain,Breast,Colon,Esophagus,Heart,Liver,Lung,Muscle,Nerve,Ovary,Pancreas,Pituitary,Prostate,Skin,Small Intestine,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,num.affected.tissues
0,A2M,104300,ALZHEIMER DISEASE,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,A4GALT,111400,"BLOOD GROUP, P1PK SYSTEMP(1) PHENOTYPE, INCLUDED",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [16]:
disease_list = disease_list.drop(['num.affected.tissues'],axis=1)
disease_list = disease_list.melt(id_vars=['gene', 'mim.ids', 'mim.titles'],value_vars = ['Adipose Tissue', 'Adrenal Gland',
       'Blood', 'Blood Vessel', 'Brain', 'Breast', 'Colon', 'Esophagus',
       'Heart', 'Liver', 'Lung', 'Muscle', 'Nerve', 'Ovary', 'Pancreas',
       'Pituitary', 'Prostate', 'Skin', 'Small Intestine', 'Spleen', 'Stomach',
       'Testis', 'Thyroid', 'Uterus', 'Vagina'])
disease_list = disease_list.rename(index=str, columns={'variable': 'Tissue', 'value':'Pos','gene': 'Description'})
disease_list.head()

,Description,mim.ids,mim.titles,Tissue,Pos
0,A2M,104300,ALZHEIMER DISEASE,Adipose Tissue,0
1,A4GALT,111400,"BLOOD GROUP, P1PK SYSTEMP(1) PHENOTYPE, INCLUDED",Adipose Tissue,0
2,AAAS,231550,ACHALASIA-ADDISONIANISM-ALACRIMA SYNDROME,Adipose Tissue,0
3,AAGAB,148600,KERATOSIS PALMOPLANTARIS PAPULOSA,Adipose Tissue,0
4,AARS,613287|616339,"CHARCOT-MARIE-TOOTH DISEASE, AXONAL, TYPE 2N|E...",Adipose Tissue,0


In [17]:
poses = disease_list['Pos'].tolist()
pos_new = []

for i in poses:
    if i == 0:
        pos_new.append(0)
    else:
        pos_new.append(1)
        
disease_list['Pos'] = pos_new

In [19]:
disease_list = disease_list[['Description','Tissue','Pos']]
gt_param_df = pd.merge(gt_param_df, disease_list,  how='left', 
                       left_on=['Description','Tissue'], right_on = ['Description','Tissue'])
print(gt_param_df.shape)
gt_param_df.head(2)

(453455, 24)


,Gene,Tissue,MeanExp,SDofExp,ZScore,MedianExp,Breadth,CV,MAD,Clustering_coeff,IsKinase,IsTF,NeighborList,NumOfKinaseNbs,NumOfNeighbors,NumOfTFNeighbors,Centrality_coeff,NeighborMeanBreadth,NeighborMeanZscore,NeighborMedianBreadth,NeighborMedianZscore,Description,Level_enc,Pos
0,ENSG00000187634,Stomach,4.720096,4.267317,-0.036851,3.423000,0.763231,0.904074,2.489289,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,7,2,0.000001,0.296060,-0.366241,0.261838,-0.380078,SAMD11,-1.0,NaN
1,ENSG00000188976,Stomach,44.847466,15.854806,-0.412520,42.380001,0.220056,0.353527,14.366420,0.253545,0,0,"['ENSG00000111676', 'ENSG00000166333', 'ENSG00...",2,110,15,0.000094,0.349456,-0.245143,0.309192,-0.260509,NOC2L,0.0,NaN


In [21]:
gt_param_df[gt_param_df['Pos']!=1].shape

(445343, 24)

In [16]:
gt_param_df.isna().sum()

Gene                          0
Tissue                        0
MeanExp                       0
SDofExp                       0
ZScore                        0
MedianExp                     0
Breadth                       0
CV                            0
MAD                           0
Clustering_coeff              0
IsKinase                      0
IsTF                          0
NeighborList                  0
NumOfKinaseNbs                0
NumOfNeighbors                0
NumOfTFNeighbors              0
Centrality_coeff              0
NeighborMeanBreadth           0
NeighborMeanZscore            0
NeighborMedianBreadth         0
NeighborMedianZscore          0
Description                   0
Level_enc                     0
Pos                      390297
dtype: int64

In [17]:
gt_param_df = gt_param_df.fillna(value=0)

In [18]:
gt_param_df.head()

,Gene,Tissue,MeanExp,SDofExp,ZScore,MedianExp,Breadth,CV,MAD,Clustering_coeff,IsKinase,IsTF,NeighborList,NumOfKinaseNbs,NumOfNeighbors,NumOfTFNeighbors,Centrality_coeff,NeighborMeanBreadth,NeighborMeanZscore,NeighborMedianBreadth,NeighborMedianZscore,Description,Level_enc,Pos
0,ENSG00000187634,Stomach,4.720096,4.267317,-0.036851,3.423000,0.763231,0.904074,2.489289,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,7,2,1.209951e-06,0.296060,-0.366241,0.261838,-0.380078,SAMD11,-1.00,0.0
1,ENSG00000188976,Stomach,44.847466,15.854806,-0.412520,42.380001,0.220056,0.353527,14.366420,0.253545,0,0,"['ENSG00000111676', 'ENSG00000166333', 'ENSG00...",2,110,15,9.446877e-05,0.349456,-0.245143,0.309192,-0.260509,NOC2L,0.00,0.0
2,ENSG00000187961,Stomach,11.037735,4.394696,-0.145059,10.360000,0.445682,0.398152,3.558246,0.333333,0,0,"['ENSG00000159251', 'ENSG00000036257', 'ENSG00...",0,4,0,2.254576e-07,0.300836,-0.350367,0.274373,-0.335166,KLHL17,0.75,0.0
3,ENSG00000187583,Stomach,1.643722,1.750657,-0.314139,1.138000,0.682451,1.065057,1.040194,0.000000,0,0,"['ENSG00000185002', 'ENSG00000110031', 'ENSG00...",0,3,1,5.063642e-07,0.470752,0.818392,0.284123,-0.294220,PLEKHN1,-1.00,0.0
4,ENSG00000187642,Stomach,0.612194,0.432204,-0.298348,0.489900,0.454039,0.705991,0.274874,0.000000,0,0,['ENSG00000160789'],0,1,0,0.000000e+00,0.311978,-0.253442,0.311978,-0.253442,PERM1,0.00,0.0


In [19]:
# know for sure pos
print(gt_param_df[(gt_param_df['Pos']>=1)&(gt_param_df['Level_enc']>0)].shape)
# conflicting positives
print(gt_param_df[(gt_param_df['Pos']>=1)&(gt_param_df['Level_enc']==0)].shape)
# no info from HPA positives
print(gt_param_df[(gt_param_df['Pos']>=1)&(gt_param_df['Level_enc']<0)].shape)

#  genes are expressed but no idea if pos or neg
print(gt_param_df[(gt_param_df['Pos']==0)&(gt_param_df['Level_enc']>0)].shape)
# most likely negatives
print(gt_param_df[(gt_param_df['Pos']==0)&(gt_param_df['Level_enc']==0)].shape)
# no info on these at all
print(gt_param_df[(gt_param_df['Pos']==0)&(gt_param_df['Level_enc']<0)].shape)

(3795, 24)
(1300, 24)
(3017, 24)
(146113, 24)
(66250, 24)
(232980, 24)


In [21]:
label_list = []

for index,row in gt_param_df.iterrows():
    if (row['Pos']>=1)&(row['Level_enc']>0) :
        label_list.append('P')
    elif (row['Pos']>=1)&(row['Level_enc']==0):
        label_list.append('U')
    elif (row['Pos']>=1)&(row['Level_enc']<0):
        label_list.append('P')
    elif (row['Pos']==0)&(row['Level_enc']>0) :
        label_list.append('U')
    elif (row['Pos']==0)&(row['Level_enc']==0):
        label_list.append('N')
    else:
        label_list.append('U')
        
print(len(label_list))
        
        


453455


In [22]:
gt_param_df['Label'] = label_list
gt_param_df[gt_param_df['Pos']>0].tail()

,Gene,Tissue,MeanExp,SDofExp,ZScore,MedianExp,Breadth,CV,MAD,Clustering_coeff,IsKinase,IsTF,NeighborList,NumOfKinaseNbs,NumOfNeighbors,NumOfTFNeighbors,Centrality_coeff,NeighborMeanBreadth,NeighborMeanZscore,NeighborMedianBreadth,NeighborMedianZscore,Description,Level_enc,Pos,Label
452835,ENSG00000102125,Blood,46.770527,19.837463,0.331103,43.820000,0.685684,0.424145,18.295313,0.022161,0,0,"['ENSG00000108953', 'ENSG00000065054', 'ENSG00...",3,86,2,0.000650,0.206173,-0.475296,0.107104,-0.602253,TAZ,-1.0,1.0,P
452844,ENSG00000160211,Blood,124.741547,81.330887,1.687938,110.599998,0.883746,0.651995,84.211800,0.131360,0,0,"['ENSG00000160211', 'ENSG00000091831', 'ENSG00...",2,49,2,0.000050,0.423327,0.106231,0.337460,-0.196223,G6PD,-1.0,1.0,P
452845,ENSG00000269335,Blood,7.418066,3.868170,1.127710,6.547000,0.749193,0.521453,3.577519,0.040846,0,0,"['ENSG00000067182', 'ENSG00000124151', 'ENSG00...",25,313,22,0.001889,0.329262,-0.165386,0.217438,-0.422435,IKBKG,-1.0,1.0,P
452847,ENSG00000130826,Blood,27.604408,41.039795,-0.105027,8.828000,0.194833,1.486712,6.413738,0.194084,0,0,"['ENSG00000254772', 'ENSG00000124783', 'ENSG00...",1,153,14,0.000118,0.259992,-0.425875,0.199139,-0.500588,DKC1,-1.0,1.0,P
452849,ENSG00000185010,Blood,1.143122,0.872557,-0.771261,0.913700,0.003229,0.763310,0.803719,0.028571,0,0,"['ENSG00000184500', 'ENSG00000126218', 'ENSG00...",0,17,1,0.000012,0.241456,-0.459097,0.198062,-0.639336,F8,-1.0,1.0,P


In [23]:
gt_param_df.shape

(453455, 25)

In [24]:
gt_param_df.to_csv('./data_generated/gt_param_df_wlabel.csv',index = False)

In [28]:
gt_param_df.head()

,Gene,Tissue,MeanExp,SDofExp,ZScore,MedianExp,Breadth,CV,MAD,Clustering_coeff,IsKinase,IsTF,NeighborList,NumOfKinaseNbs,NumOfNeighbors,NumOfTFNeighbors,Centrality_coeff,NeighborMeanBreadth,NeighborMeanZscore,NeighborMedianBreadth,NeighborMedianZscore,Description,Level_enc,Pos,Label
0,ENSG00000187634,Stomach,4.720096,4.267317,-0.036851,3.423000,0.763231,0.904074,2.489289,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0,7,2,1.209951e-06,0.296060,-0.366241,0.261838,-0.380078,SAMD11,-1.00,0.0,U
1,ENSG00000188976,Stomach,44.847466,15.854806,-0.412520,42.380001,0.220056,0.353527,14.366420,0.253545,0,0,"['ENSG00000111676', 'ENSG00000166333', 'ENSG00...",2,110,15,9.446877e-05,0.349456,-0.245143,0.309192,-0.260509,NOC2L,0.00,0.0,N
2,ENSG00000187961,Stomach,11.037735,4.394696,-0.145059,10.360000,0.445682,0.398152,3.558246,0.333333,0,0,"['ENSG00000159251', 'ENSG00000036257', 'ENSG00...",0,4,0,2.254576e-07,0.300836,-0.350367,0.274373,-0.335166,KLHL17,0.75,0.0,U
3,ENSG00000187583,Stomach,1.643722,1.750657,-0.314139,1.138000,0.682451,1.065057,1.040194,0.000000,0,0,"['ENSG00000185002', 'ENSG00000110031', 'ENSG00...",0,3,1,5.063642e-07,0.470752,0.818392,0.284123,-0.294220,PLEKHN1,-1.00,0.0,U
4,ENSG00000187642,Stomach,0.612194,0.432204,-0.298348,0.489900,0.454039,0.705991,0.274874,0.000000,0,0,['ENSG00000160789'],0,1,0,0.000000e+00,0.311978,-0.253442,0.311978,-0.253442,PERM1,0.00,0.0,N


### Normalizing the features

In [ ]:
features = ['MeanExp','SDofExp','ZScore','MedianExp','CV',
            'MAD','NumOfKinaseNbs','NumOfNeighbors', 'NumOfTFNeighbors',
           'NeighborMeanBreadth','NeighborMeanZscore']

for feature in features:
    normalized_list = []
    tissue_max_vals =  gt_param_df.groupby('Tissue')[feature].max()
    tissue_max_vals = tissue_max_vals.to_dict()
    print(feature)
    print(datetime.datetime.now())
    count = 0
    for index,row in gt_param_df.iterrows():
        count+=1
        feature_val = row[feature]
        tissue = row['Tissue']
        feature_val = feature_val/tissue_max_vals[tissue]
        normalized_list.append(feature_val)
        if count % 10000 ==0:
            print(count)
            
    gt_param_df[feature] = normalized_list



In [25]:
gt_param_df.to_csv('./data_generated/gt_param_df_wlabel_norm.csv',index = False)

In [32]:
gt_param_df[gt_param_df['Gene']=='ENSG00000187634'].head()

,Gene,Tissue,MeanExp,SDofExp,ZScore,MedianExp,Breadth,CV,MAD,Clustering_coeff,IsKinase,IsTF,NeighborList,NumOfKinaseNbs,NumOfNeighbors,NumOfTFNeighbors,Centrality_coeff,NeighborMeanBreadth,NeighborMeanZscore,NeighborMedianBreadth,NeighborMedianZscore,Description,Level_enc,Pos,Label
0,ENSG00000187634,Stomach,0.000106,0.000100,-0.006795,0.000085,0.763231,0.059735,0.000054,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0.0,0.003605,0.009132,1.209951e-06,0.296060,-0.070267,0.261838,-0.380078,SAMD11,-1.0,0.0,U
92,ENSG00000187634,Prostate,0.000539,0.001144,0.283249,0.000405,0.995918,0.062393,0.000767,0.100000,0,0,"['ENSG00000121060', 'ENSG00000115239', 'ENSG00...",0.0,0.002268,0.009091,6.463013e-07,0.765714,0.088690,0.844898,0.559634,SAMD11,-1.0,0.0,U
187,ENSG00000187634,Salivary Gland,0.000027,0.000025,-0.055231,0.000023,0.111111,0.071500,0.000014,0.047619,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0.0,0.003481,0.008772,1.279273e-06,0.320106,-0.035841,0.308642,-0.312886,SAMD11,-1.0,0.0,U
285,ENSG00000187634,Testis,0.000197,0.000231,0.037531,0.000196,0.997230,0.028482,0.000213,0.100000,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0.0,0.003338,0.009434,7.092571e-07,0.541828,0.059871,0.529086,-0.173701,SAMD11,-1.0,0.0,U
378,ENSG00000187634,Brain,0.000023,0.000074,-0.181917,0.000013,0.222180,0.032268,0.000025,0.000000,0,0,"['ENSG00000128591', 'ENSG00000121060', 'ENSG00...",0.0,0.001697,0.004292,5.735608e-07,0.283497,-0.112089,0.169758,-0.427209,SAMD11,-1.0,0.0,U


# Adding labels to gtex params

In [43]:
gtex_params = pickle.load(open('/fs/cbcb-scratch/gowthami/GTF/GTF_Project_v2/data/gene_tissue_parameter_df.p','rb'))
gtex_params.head(2)

,G_T,Gene,Tissue,MeanExp,SDofExp,ZScore,MinExp,MaxExp,MedianExp,Breadth,CoeffOfVar,MedAbsDev
0,A1BG_Adipose Tissue,A1BG,Adipose Tissue,3.408423,2.116964,-0.121740,0.4395,27.86,2.9600,0.490590,0.621098,1.430711
1,A1BG_Adrenal Gland,A1BG,Adrenal Gland,1.605872,1.084509,-0.150274,0.2848,10.69,1.3755,0.073684,0.675340,0.573989


In [44]:
gtex_params.shape

(226380, 12)

In [45]:
gtex_params_label = pd.merge(gtex_params, disease_list,  how='left', 
                       left_on=['Gene','Tissue'], right_on = ['Gene','Tissue'])
print(gtex_params_label.shape)
gtex_params_label.head(2)
  

(226380, 13)


,G_T,Gene,Tissue,MeanExp,SDofExp,ZScore,MinExp,MaxExp,MedianExp,Breadth,CoeffOfVar,MedAbsDev,Pos
0,A1BG_Adipose Tissue,A1BG,Adipose Tissue,3.408423,2.116964,-0.121740,0.4395,27.86,2.9600,0.490590,0.621098,1.430711,NaN
1,A1BG_Adrenal Gland,A1BG,Adrenal Gland,1.605872,1.084509,-0.150274,0.2848,10.69,1.3755,0.073684,0.675340,0.573989,NaN


In [46]:
gtex_params_label.isnull().any()


G_T           False
Gene          False
Tissue        False
MeanExp       False
SDofExp       False
ZScore         True
MinExp        False
MaxExp        False
MedianExp     False
Breadth       False
CoeffOfVar     True
MedAbsDev     False
Pos            True
dtype: bool

In [47]:
gtex_params_label = gtex_params_label[np.isfinite(gtex_params_label['ZScore'])]
gtex_params_label = gtex_params_label[np.isfinite(gtex_params_label['CoeffOfVar'])]
gtex_params_label = gtex_params_label.fillna(value=0)

In [48]:
gtex_params_label.to_csv('/scratch1/gowthami/GTF/GTF_Project_v2/data/gtex_params_label.csv',index = False)